In [300]:
import logging
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os
from scipy.optimize import curve_fit

In [301]:
# Logging setup
logging.basicConfig(level=logging.NOTSET)
logger = logging.getLogger()

In [302]:
# Reading FITS file
logger.info('Opening FITS file...')
image = fits.open("D:\\Files\\differential_photometry group2\\stars.fits")[0].data
logger.info('Opened successfully')

# Note that the stars.fits file is already in a 2D format.

INFO:root:opening fits file...
INFO:root:opened successfully


In [303]:
# Define function to get cropped image around the center of mass
def get_croped_image(image, center_of_mass):
    i_center_of_mass = center_of_mass[0]
    j_center_of_mass = center_of_mass[1]
    croped_image = np.zeros((31, 31), dtype=int)
    i_counter = 0
    for i in range(i_center_of_mass - 15, i_center_of_mass + 16):
        j_counter = 0
        for j in range(j_center_of_mass - 15, j_center_of_mass + 16):
            croped_image[i_counter][j_counter] = image[i][j]
            j_counter += 1
        i_counter += 1
    return croped_image

# Define function to calculate center of mass of a star
def get_center_of_mass(image, i_of_star, j_of_star):
    total_mass = 0
    weight_vector = np.array([0., 0.])
    for i in range(i_of_star - 20, i_of_star + 21):
        for j in range(j_of_star - 20, j_of_star + 21):
            weight_vector += image[i][j] * np.array([i, j])
            total_mass += image[i][j]
    center_of_mass = (np.round(weight_vector / total_mass)).astype(int)
    return center_of_mass, get_croped_image(image, center_of_mass)

In [304]:
# Define function to calculate Full Width at Half Maximum (FWHM) of a star
def get_FWHM(data):
    max_data = np.max(data)
    half_max_data = (max_data / 2).astype(int)
    indices = np.where(data >= half_max_data)
    x_indices = indices[0]
    y_indices = indices[1]
    FWHM_x = (x_indices.max() - x_indices.min())
    FWHM_y = (y_indices.max() - y_indices.min())
    FWHM = (FWHM_x + FWHM_y) / 2
    return int(FWHM)

# Define function to calculate 1D FWHM
def get_FWHM_1D(data):
    max_data = np.max(data)
    half_max_data = (max_data / 2).astype(int)
    indices = np.where(data >= half_max_data)
    x_indices = indices[0]
    FWHM = (x_indices.max() - x_indices.min())
    return int(FWHM)

In [305]:
# Define function to calculate signal within a given radius around the star
def calculate_signal(image, i_of_star, j_of_star, radius):
    signal = 0
    pixel_counter = 0
    for i in range(i_of_star - radius, i_of_star + radius + 1):
        for j in range(j_of_star - radius, j_of_star + radius + 1):
            x_val = i - i_of_star
            y_val = j - j_of_star
            if (x_val ** 2 + y_val ** 2 <= radius ** 2):
                signal += image[i][j]
                pixel_counter += 1
    return signal / pixel_counter

In [306]:
# Define function to calculate noise within a given radius around the star
def calculate_noise(image, i_of_star, j_of_star, radius):
    noise = 0
    pixel_counter = 0
    big_radius = np.round(radius * 2.5).astype(int)
    for i in range(i_of_star - big_radius, i_of_star + big_radius + 1):
        for j in range(j_of_star - big_radius, j_of_star + big_radius + 1):
            x_val = i - i_of_star
            y_val = j - j_of_star
            if ((x_val ** 2 + y_val ** 2 >= (2 * radius) ** 2) and (x_val ** 2 + y_val ** 2 <= (2.5 * radius) ** 2)):
                noise += image[i][j]
                pixel_counter += 1
    return noise / pixel_counter

In [307]:
center_of_mass, croped_image = get_center_of_mass(image, 790, 426)
radius = np.round(get_FWHM(croped_image)).astype(int)
signal = calculate_signal(image, center_of_mass[0], center_of_mass[1], (radius).astype(int))
radius = np.round(get_FWHM_1D(croped_image[15])).astype(int)
radius

24

In [308]:
noise = calculate_noise(image, center_of_mass[0], center_of_mass[1], radius)
noise

3394.440196078431

In [309]:
SNR = signal/noise
SNR

3.4675220925641206

In [314]:
# Define positions of stars and their magnitudes
guessed_center_of_mass = {
    "base": [790, 426],
    "first": [1403, 717],
    "second": [1476, 680],
    "third": [1565, 640],
    "fourth": [625, 1264]
}
m0 = 8.88  # Reference magnitude
F0 = 0  # Initial flux

# Iterate over each star and calculate its magnitude and signal-to-noise ratio
for key in guessed_center_of_mass.keys():
    position = guessed_center_of_mass[key]
    center_of_mass, croped_image = get_center_of_mass(image, position[0], position[1])
    radius = np.round(get_FWHM_1D(croped_image[0]) / 2).astype(int)
    print(f'radius = {radius}')
    signal = calculate_signal(image, center_of_mass[0], center_of_mass[1], radius)
    noise = calculate_noise(image, center_of_mass[0], center_of_mass[1], radius)
    if (key == "base"):
        F0 = calculate_signal(croped_image - noise, 15, 15, radius)
    else:
        F1 = calculate_signal(croped_image - noise, 15, 15, radius)
        magnitude = -2.5 * np.log10(F1 / F0) + m0
        SNR = signal / noise
        sigma_magnitude = 1.0875 / SNR
        print(f"magnitude of {key} = {magnitude} +- {sigma_magnitude} -- SNR = {SNR}")

radius = 8
radius = 13
magnitude of first = 9.934814461981611 +- 0.14243225353622951 -- SNR = 7.635208830866247
radius = 7
magnitude of second = 9.711105955923749 +- 0.11463631871226405 -- SNR = 9.486522353614768
radius = 13
magnitude of third = 10.056649867268103 +- 0.1559748106109688 -- SNR = 6.972279663236356
radius = 10
magnitude of forth = 11.408366103823877 +- 0.4866419579347575 -- SNR = 2.2347024999965117
